In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ibrahimserouis99/one-piece-image-classifier

Dataset URL: https://www.kaggle.com/datasets/ibrahimserouis99/one-piece-image-classifier
License(s): GPL-2.0
one-piece-image-classifier.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip one-piece-image-classifier.zip

Archive:  one-piece-image-classifier.zip
replace Data/Data/Ace/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install tensorflow keras

In [ ]:
train_dir = '/content/Data/Data'
validation_dir = '/content/one-piece-image-classifier/validation'

In [56]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# กำหนดพาธของข้อมูล
train_dir = '/content/Data/Data'
validation_dir = '/content/Data/Data'

# สร้าง ImageDataGenerator สำหรับการ augment ข้อมูล
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# โหลดข้อมูล
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 1964 images belonging to 3 classes.
Found 1964 images belonging to 3 classes.


In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# สร้างโมเดล
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 คือจำนวนคลาส
])

# คอมไพล์โมเดล
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# แสดงโครงสร้างโมเดล
model.summary()

# เทรนโมเดล
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,035,203 (72.61 MB)

 Trainable params: 19,035,203 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - accuracy: 0.3963 - loss: 1.5643 - val_accuracy: 0.5615 - val_loss: 1.0872
Epoch 2/30
 1/61 ━━━━━━━━━━━━━━━━━━━━ 45s 766ms/step - accuracy: 0.5833 - loss: 0.9789

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 703ms/step - accuracy: 0.5833 - loss: 0.9789 - val_accuracy: 0.4985 - val_loss: 1.4396
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.5986 - loss: 0.9353 - val_accuracy: 0.6808 - val_loss: 0.9152
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 33s 520ms/step - accuracy: 0.5625 - loss: 0.9144 - val_accuracy: 0.6404 - val_loss: 0.9923
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - accuracy: 0.6782 - loss: 0.8157 - val_accuracy: 0.7290 - val_loss: 0.7781
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 41s 674ms/step - accuracy: 0.7500 - loss: 0.8862 - val_accuracy: 0.7172 - val_loss: 0.7313
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 203s 3s/step - accuracy: 0.6811 - loss: 0.7351 - val_accuracy: 0.7290 - val_loss: 0.6628
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 692ms/step - accuracy: 0.7188 - loss: 0.6626 - val_accuracy: 0.7228 - val_loss: 0.6710
Epoch 9/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.7371 - loss: 0.6466 - val_accuracy: 0.7377 - val_

In [58]:
model.save('one_piece_character_classifier.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')